<a href="https://colab.research.google.com/github/Mnozzi/online-shopping-mall/blob/main/Try%26learn1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
auth.authenticate_user()

!pip install PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [2]:
fileId = drive.CreateFile({'id': '1bjsH1wijxXmXH1quyEpOW1vLgPCZIFZ3'}) #DRIVE_FILE_ID is file id example: 1iytA1n2z4go3uVCwE_vIKouTKyIDjEq
print(fileId['title'])  # folder_data.zip
fileId.GetContentFile('dataset.zip')  # Save Drive file as a local file

!unzip dataset.zip -d ./

Dataset.zip
Archive:  dataset.zip
   creating: ./Dataset/
   creating: ./Dataset/Train/
   creating: ./Dataset/Train/gl/
  inflating: ./Dataset/Train/gl/Im256.jpg  
  inflating: ./Dataset/Train/gl/Im257.jpg  
  inflating: ./Dataset/Train/gl/Im258.jpg  
  inflating: ./Dataset/Train/gl/Im259.jpg  
  inflating: ./Dataset/Train/gl/Im260.jpg  
  inflating: ./Dataset/Train/gl/Im261.jpg  
  inflating: ./Dataset/Train/gl/Im262.jpg  
  inflating: ./Dataset/Train/gl/Im263.jpg  
  inflating: ./Dataset/Train/gl/Im264.jpg  
  inflating: ./Dataset/Train/gl/Im265.jpg  
  inflating: ./Dataset/Train/gl/Im266.jpg  
  inflating: ./Dataset/Train/gl/Im267.jpg  
  inflating: ./Dataset/Train/gl/Im268.jpg  
  inflating: ./Dataset/Train/gl/Im269.jpg  
  inflating: ./Dataset/Train/gl/Im270.jpg  
  inflating: ./Dataset/Train/gl/Im271.jpg  
  inflating: ./Dataset/Train/gl/Im272.jpg  
  inflating: ./Dataset/Train/gl/Im273.jpg  
  inflating: ./Dataset/Train/gl/Im274.jpg  
  inflating: ./Dataset/Train/gl/Im275.jpg  

In [5]:
import tensorflow as tf


In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate, GlobalAveragePooling2D
from keras.layers import BatchNormalization, Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras import optimizers
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from imgaug import augmenters as iaa
from keras.utils import plot_model
import numpy as np

# channels last, no. of channels = 3
img_width, img_height = 256, 256
input_shape = (img_width, img_height, 3)

train_data_dir = "Dataset/Train"
validation_data_dir = "Dataset/Validation"
test_data_dir = "Dataset/Test"
nb_train_samples = 280
nb_validation_samples = 80
n_test_samples = 40
epochs = 100

In [7]:
def get_steps(n_samples, batch_size):
    if n_samples <= batch_size:
        return n_samples
    elif np.remainder(n_samples, batch_size) == 0:
        return n_samples//batch_size
    else:
        return n_samples//batch_size + 1

#MODEL

In [8]:
def block_encoder(layer, filters, kernel_size, activation):
  block = Conv2D(filters, kernel_size, padding='same', data_format="channels_last", activation = activation)(layer)
  block = Conv2D(filters * 2, kernel_size, padding='same', data_format="channels_last", activation = activation)(block)
  block = MaxPooling2D(pool_size = (2, 2))(block)
  block = BatchNormalization()(block)

  print(block.shape)
  return block

In [9]:
def block_decoder(layer, filters, kernel_size, activation, upSample = True):
  block = Conv2D(filters * 2, kernel_size, padding='same', data_format="channels_last", activation = activation)(layer)
  block = Conv2D(filters, kernel_size, padding='same', data_format="channels_last", activation = activation)(block)
  if upSample:
    block = UpSampling2D(size=(2,2))(layer)
  block = BatchNormalization()(block)

  print(block.shape)
  return block

In [11]:
inputLayer = Input(shape=input_shape)

en_block1 = block_encoder(inputLayer, 2, (2, 2), "relu") # output_shape = 128, 128, 4
en_block2 = block_encoder(en_block1, 4, (2, 2), "relu") # output_shape = 64, 64, 8
de_block3 = block_decoder(en_block2, 8, (2, 2), "relu", upSample = False) # output_shape = 64, 64, 8
temp = Concatenate()([de_block3, en_block2])
de_block2 = block_decoder(temp, 4, (2, 2), "relu")
de_block1 = block_decoder(Concatenate()([de_block2, en_block1]), 2, (2, 2), "relu")

output = Conv2D(2, (1, 1), activation = "softmax")(de_block1)
model = Model(inputs=[inputLayer], outputs=[output])
model.compile(loss="categorical_crossentropy",
              optimizer=optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
              metrics=["accuracy"])

(None, 128, 128, 4)
(None, 64, 64, 8)
(None, 64, 64, 8)
(None, 128, 128, 16)
(None, 256, 256, 20)


In [12]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 256, 256, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_11 (Conv2D)        │ (None, 256, 256, 2)    │             26 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_12 (Conv2D)        │ (None, 256, 256, 4)    │             36 │ conv2d_11[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_2           │ (None, 128, 128, 4)    │              0 │ conv2d_12[0][0]        │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_5     │ (None, 128, 128, 4)    │             16 │ max_pooling2d_2[0][0]  │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_13 (Conv2D)        │ (None, 128, 128, 4)    │             68 │ batch_normalization_5… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_14 (Conv2D)        │ (None, 128, 128, 8)    │            136 │ conv2d_13[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_3           │ (None, 64, 64, 8)      │              0 │ conv2d_14[0][0]        │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_6     │ (None, 64, 64, 8)      │             32 │ max_pooling2d_3[0][0]  │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_15 (Conv2D)        │ (None, 64, 64, 16)     │            528 │ batch_normalization_6… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_16 (Conv2D)        │ (None, 64, 64, 8)      │            520 │ conv2d_15[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_7     │ (None, 64, 64, 8)      │             32 │ conv2d_16[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_2             │ (None, 64, 64, 16)     │              0 │ batch_normalization_7… │
│ (Concatenate)             │                        │                │ batch_normalization_6… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ up_sampling2d_2           │ (None, 128, 128, 16)   │              0 │ concatenate_2[0][0]    │
│ (UpSampling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_8     │ (None, 128, 128, 16)   │             64 │ up_sampling2d_2[0][0]  │
│ (BatchNormalization)      │                        │                │                        │
├──────────────────────

 Total params: 1,580 (6.17 KB)

 Trainable params: 1,468 (5.73 KB)

 Non-trainable params: 112 (448.00 B)

In [15]:
input = Input(shape=input_shape)

block1 = BatchNormalization(name='norm_0')(input)

# Block 1
block1 = Conv2D(8, (3,3), name='conv_11', activation='relu')(block1)
block1 = Conv2D(16, (3,3), name='conv_12', activation='relu')(block1)
block1 = Conv2D(32, (3,3), name='conv_13', activation='relu')(block1)
block1 = Conv2D(64, (3,3), name='conv_14', activation='relu')(block1)
block1 = MaxPooling2D(pool_size=(2, 2))(block1)
block1 = BatchNormalization(name='norm_1')(block1)

block1 = Conv2D(16, 1)(block1)

# Block 2
block2 = Conv2D(32, (3,3), name='conv_21', activation='relu')(block1)
block2 = Conv2D(64, (3,3), name='conv_22', activation='relu')(block2)
block2 = Conv2D(64, (3,3), name='conv_23', activation='relu')(block2)
block2 = Conv2D(128, (3,3), name='conv_24', activation='relu')(block2)
block2 = MaxPooling2D(pool_size=(2, 2))(block2)
block2 = BatchNormalization(name='norm_2')(block2)

block2 = Conv2D(64, 1)(block2)

# Block 3
block3 = Conv2D(64, (3,3), name='conv_31', activation='relu')(block2)
block3 = Conv2D(128, (3,3), name='conv_32', activation='relu')(block3)
block3 = Conv2D(128, (3,3), name='conv_33', activation='relu')(block3)
block3 = Conv2D(64, (3,3), name='conv_34', activation='relu')(block3)
block3 = MaxPooling2D(pool_size=(2, 2))(block3)
block3 = BatchNormalization(name='norm_3')(block3)

# Block 4
block4 = Conv2D(64, (3,3), name='conv_41', activation='relu')(block3)
block4 = Conv2D(32, (3,3), name='conv_42', activation='relu')(block4)
block4 = Conv2D(16, (3,3), name='conv_43', activation='relu')(block4)
block4 = Conv2D(8, (2,2), name='conv_44', activation='relu')(block4)
block4 = MaxPooling2D(pool_size=(2, 2))(block4)
block4 = BatchNormalization(name='norm_4')(block4)

block4 = Conv2D(2, 1)(block4)

block5 = GlobalAveragePooling2D()(block4)
output = Activation('softmax')(block5)

model = Model(inputs=[input], outputs=[output])
model.compile(loss="categorical_crossentropy",
              optimizer=optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
              metrics=["accuracy"])

plot_model(model, to_file = "model.png", show_shapes = True)
model.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)           │ (None, 256, 256, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ norm_0 (BatchNormalization)          │ (None, 256, 256, 3)         │              12 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_11 (Conv2D)                     │ (None, 254, 254, 8)         │             224 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_12 (Conv2D)                     │ (None, 252, 252, 16)        │           1,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_13 (Conv2D)                     │ (None, 250, 250, 32)        │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_14 (Conv2D)                     │ (None, 248, 248, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_12 (MaxPooling2D)      │ (None, 124, 124, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ norm_1 (BatchNormalization)          │ (None, 124, 124, 64)        │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_28 (Conv2D)                   │ (None, 124, 124, 16)        │           1,040 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_21 (Conv2D)                     │ (None, 122, 122, 32)        │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_22 (Conv2D)                     │ (None, 120, 120, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_23 (Conv2D)                     │ (None, 118, 118, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_24 (Conv2D)                     │ (None, 116, 116, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_13 (MaxPooling2D)      │ (None, 58, 58, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ norm_2 (BatchNormalization)          │ (None, 58, 58, 128)         │             512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_29 (Conv2D)                   │ (None, 58, 58, 64)          │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_31 (Conv2D)                     │ (None, 56, 56, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_32 (Conv2D)                     │ (None, 54, 54, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_33 (Conv2D)                     │ (None, 52, 52, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_34 (Conv2D)                     │ (None, 50, 50, 64)          │          73,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_14 (MaxPooling2D)      │ (None, 25, 25, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 561,526 (2.14 MB)

 Trainable params: 560,992 (2.14 MB)

 Non-trainable params: 534 (2.09 KB)

In [16]:
# Augmentationb
sometimes = lambda aug: iaa.Sometimes(0.6, aug)
seq = iaa.Sequential([
                      iaa.GaussianBlur(sigma=(0 , 1.0)),
                      iaa.Sharpen(alpha=1, lightness=0),
                      iaa.CoarseDropout(p=0.1, size_percent=0.15),
                              sometimes(iaa.Affine(
                                                    scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
                                                    translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
                                                    rotate=(-30, 30),
                                                    shear=(-16, 16)))
                    ])

# Data Generators

In [27]:
# Data generators

train_datagen = ImageDataGenerator(
    rescale=1./255,
    # preprocessing_function=seq.augment_image,
    horizontal_flip=True,
    vertical_flip=True)

val_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    vertical_flip=True)

test_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    vertical_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode="categorical")



validation_generator = val_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    class_mode="categorical")

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    class_mode="categorical")

Found 280 images belonging to 2 classes.
Found 80 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


In [33]:
import tensorflow as tf

def train_generator():
    # ... 加载图像数据 ...
    train_x=
    train_y=
    dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y))
    dataset = dataset.map(lambda x, y: (tf.image.resize(x, (128, 128)), y))
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

    return dataset

In [30]:
import tensorflow as tf

# 加载图像数据
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_data_dir,
    labels='inferred',
    label_mode='categorical',
    image_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=True
)

# 预处理数据
def preprocess(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    # ... 其他预处理操作 ...
    return image, label

train_dataset = train_dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)

# 优化性能
train_dataset = train_dataset.cache()
train_dataset = train_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)



Found 280 files belonging to 2 classes.


In [21]:
# Callbacks
checkpoint = ModelCheckpoint("f1.keras", monitor='acc', verbose=1, save_best_only=True,
                             save_weights_only=False, mode='auto', save_freq=1)  # 修改文件路径
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=2, verbose=0,
                              mode='auto', cooldown=0, min_lr=0)

# Training Model

In [34]:
batch_size = 32
model.fit(
    train_generator(),
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    callbacks=[checkpoint, reduce_lr]
)

NameError: name 'x' is not defined

# Prediction on Test Data

In [ ]:
# Callbacks
checkpoint = ModelCheckpoint("f1.h5", monitor='acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=2, verbose=0, mode='auto', cooldown=0, min_lr=0)

In [ ]:
batch_size = 10
pred = model.predict_generator(test_generator,
                               steps=get_steps(n_test_samples, batch_size),
                               callbacks=[checkpoint, reduce_lr],
                               verbose=1)

4/4 [==============================] - 2s 408ms/step


In [ ]:
pred

array([[0.66776055, 0.33223945],
       [0.39407468, 0.6059253 ],
       [0.31803977, 0.6819602 ],
       [0.816509  , 0.183491  ],
       [0.66935116, 0.3306488 ],
       [0.49172097, 0.508279  ],
       [0.5092242 , 0.49077582],
       [0.2921776 , 0.70782244],
       [0.43954197, 0.56045806],
       [0.74468166, 0.2553184 ],
       [0.85571307, 0.14428699],
       [0.5027314 , 0.4972686 ],
       [0.02267935, 0.9773207 ],
       [0.79438525, 0.20561472],
       [0.05778668, 0.9422133 ],
       [0.04669693, 0.95330304],
       [0.5223356 , 0.47766447],
       [0.5796036 , 0.42039636],
       [0.7943506 , 0.20564936],
       [0.37829894, 0.6217011 ],
       [0.47306046, 0.5269396 ],
       [0.46297646, 0.5370236 ],
       [0.4420952 , 0.55790484],
       [0.7734003 , 0.22659972],
       [0.02141242, 0.97858757],
       [0.44251254, 0.5574875 ],
       [0.93071204, 0.06928799],
       [0.04541377, 0.95458627],
       [0.11802074, 0.8819793 ],
       [0.66649526, 0.33350477],
       [0.

In [ ]:
threshold = 0.6
result = []
for x, y in pred:
  if x >= 0.6:
    result.append(0)
  else:
    result.append(1)

print(result)

[0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1]


In [ ]:
print(result.count(1))

52


In [ ]:
print(result.count(0))

28


In [ ]:
print(len(result))

80
